# <div  style="color:#303030;font-family:'arial blACK', sans-serif,monospace; text-align: center; padding: 50px 0; vertical-align:middle;" > <img src="https://github.com/PIA-Group/ScientIST-notebooks/blob/master/_Resources/Images/Lightbulb.png?raw=true" style=" background:#00a0e4;border-radius:10px;width:150px;text-align:left; margin-left:5%"  /> <span style="position:relative; bottom:70px; margin-left:5%">Live Streaming and Signal Processing</span> </div>

## <span style="color:#00a0e4;"> Keywords: </span>

```Serial Port```, ```Python```, ```Live Streaming```, `Signal Processing`

# I. Introduction
<br>
<div class="title"style="width:100%; background:#00a0e4;font-family:'arial black',monospace; text-align: center; padding: 7px 0; border-radius: 5px 50px;margin-top:-15px" >  </div>

## <div style="color:#00a0e4"> 1. Background </div>

Whether for post-processing, long-term storage, customized development or other reasons alike, in many biomedical applications it is often useful to interact with devices programmatically, from within our own software running on a computer or mobile phone, rather than relying on third-party tools. In this lesson we will experiment with handling of live (aka real time) data acquisition from within a set of purpose-built programs, with special emphasis on serial port communication and data logging using the Python programming language.

<img src="https://github.com/PIA-Group/ScientIST-notebooks/blob/master/_Resources/Images/C.Signal_Processing_IMG/c004/serialmonitor.png?raw=true" width="450">

## <div style="color:#00a0e4"> 2. Objectives</div>
* Get familiar with the Python programming language
* Learn the basics of programatic access to the serial port
* Create a simple serial port monitor, a live plotter and a data logger

## <div style="color:#00a0e4"> 3. Materials </div>
* Python 3
* Arduino IDE
* 1x Arduino Uno
* 1x USB cable
* 1x Breadboard
* 1x Function Generator

<div style="background:#fbb144;font-family:'arial', monospace; text-align: center; padding: 10px 0; border-radius:10px; width:70%; margin:auto " >
  <span style="font-size:20px;position:relative;color:white; ">  Warning! </span> <br>
  <div style="background:#ffd08a;font-size:12px"> 
    Unless otherwise noticed make sure that you use a 19200bps baud rate
throughout the following experiments.  
</div>

# II. Experimental
<br>
<div style="width:100%; background:#00a0e4;color:#282828;font-family:'arial black'; text-align: center; padding: 7px 0; border-radius: 5px 50px; margin-top:-15px" > </div>

This section should guide the students during their experimental procedure, and contain the most relevant content.

## <div style="color:#00a0e4;">  1. Configuring a Python Framework</div>


1. As you know by now, Python is a "batteries included" language. If you browse around there are dozens of distributions available, with different degrees of completion in what concerns the set of pre-installed modules. One of the most all-terrain options to get started is the Anaconda Python distribution.

Anaconda Python download page:

<img src="https://github.com/afonsocraposo/ScientIST-notebooks/blob/master/_Resources/Images/C.Signal_Processing_IMG/l03/anaconda.png?raw=true" width="700" style="border: 2px solid #555;">

<div style="background:#946db2;font-family:'arial', monospace; text-align: center; padding: 10px 0; border-radius:10px; width:70%; margin:auto " >
  <span style="font-size:20px;position:relative;color:white; "> Explore </span> <br>
  <div style="background:#d0b3e6;font-size:12px"> 
    <a href="https://www.anaconda.com/products/individual">https://www.anaconda.com/products/individual</a>
</div>

2. Once installed, the Anaconda Navigator application will be available in your computer. From the starting page you have several tools to choose from; we will use Spyder, so select that option to open the IDE.

        Anaconda Navigator starting page:
<img src="https://github.com/PIA-Group/ScientIST-notebooks/blob/master/_Resources/Images/C.Signal_Processing_IMG/c004/AnacondaNavigator.png?raw=true" width="550">

<div style="background:#48ba57;font-family:'arial', monospace; text-align: center; padding: 10px 0; border-radius:10px; width:70%; margin:auto " >
  <span style="font-size:20px;position:relative;color:white; ">  Note </span> <br>
  <div style="background:#9de3a6;font-size:12px"> 
      The Spyder IDE is used as an example for this laboratory. If you prefer, you can use another software of your choice.
</div>

    3. Spyder provides several convenient features for Python code editing and execution; the following figure shows the main functionalities you will need.

        Overview of the Spyder IDE Graphical User Interface (GUI):
<img src="https://github.com/PIA-Group/ScientIST-notebooks/blob/master/_Resources/Images/C.Signal_Processing_IMG/c004/SpyderIDE.png?raw=true" width="550">
<img src="https://github.com/PIA-Group/ScientIST-notebooks/blob/master/_Resources/Images/C.Signal_Processing_IMG/c004/SpyderIDEexplained.png?raw=true" width="550">

## <div style="color:#00a0e4;">  2. DiY Serial Monitor</div>


The ability to control and capture data transmitted by an embedded device in our custom software applications on the receiver (e.g. a computer), is usually a convenient step that opens up a whole new world of opportunities. As such, this experiment will focus on using Python to implement a program that sets up a communication channel between the computer and the Arduino, with the purpose of receiving a data stream sent by the latter.

1. Connect the Arduino to the Function Generator as you did in the first laboratory, *L1. Seeeduino Nano Essentials*, but to the `A0` input instead. Setup a signal of your choice taking into account the voltage limits of the Seeeduino Nano.

<img src="https://github.com/afonsocraposo/ScientIST-notebooks/blob/master/_Resources/Images/C.Signal_Processing_IMG/l03/function_generator.png?raw=true" width="600">

<div style="background:#fe9b29; font-family:'arial', monospace;text-align: center; padding: 10px 0; border-radius:10px; width:70%; margin:auto " >
  <span style="font-size:20px;position:relative;color:white; ">  Caution! </span> <br>
  <div style="background:#ffdab0;font-size:12px"> 
    The Seeeduino Nano works with low voltages and currents, which means you must be careful when connecting a function generator to it. Always make sure that the output voltage is between 0 and 5V and never above this range, since you can damage your device. Therefore, make sure the OUTPUT is turned OFF and only turn it ON once you are sure that the function generator has the appropriate settings.
</div>

2. Flash the Arduino program you used in the second laboratory, *L2. Analog-to-Digital Conversion (ADC)*.

```cpp
bool state=false;
int f=100, dt=0;
char command;
unsigned long t=0, lt=0; 

void setup() {
  // initialize the serial communication:
  Serial.begin(9600);
  dt=int(1000*1/(float)f);
}

void loop() {
  if (Serial.available()) {
    command = Serial.read();
    switch (command) {
      case 'S': 
        state=!state;
        break;
      case 'F':
        f=Serial.parseInt();
        Serial.println(f);
        dt=int(1000*1/(float)f);
        break;
    }
    lt=millis();
  }
  if (state) {
    t=millis();
    if ((t-lt)>=dt) {
      Serial.print(t);
      Serial.print(",");
      Serial.print(analogRead(A0));
      Serial.print("\n");
      lt=t;
    }
  }
}
```

3. Open the Serial Monitor tool from the Arduino IDE and confirm that the program is working as expected (i.e. streaming data based on the commands sent from the computer)

4. Create a new (empty) Python program in the Spyder IDE 

5. Write a program that uses the **PySerial** (serial) module to connect to your Arduino, start the data acquisition process (by sending the S command), continuously read and print on the console the data streamed by the Arduino for 10 seconds, and stop the data acquisition process (by resending the S command)

6. Run your program and verify the output in the interactive shell; a series of lines showing the messages transmitted by the Arduino should appear just like in the Arduino Serial Monitor

<div style="background:#fe9b29;font-family:'arial', monospace; text-align: center; padding: 10px 0; border-radius:10px; width:70%; margin:auto " >
  <span style="font-size:20px;position:relative;color:white; ">  Caution! </span> <br>
  <div style="background:#ffdab0;font-size:12px"> 
    ONLY ONE software application can be connected to the same serial
port at any given time so make sure that the Arduino Serial Monitor
(which connects to the serial port also) is closed before running your
program, otherwise you will have an error indicating that Python is not
able to open the communication port.    
</div>

<div style="background:#48ba57;font-family:'arial', monospace; text-align: center; padding: 10px 0; border-radius:10px; width:70%; margin:auto " >
  <span style="font-size:20px;position:relative;color:white; ">  Note </span> <br>
  <div style="background:#9de3a6;font-size:12px"> 
      You can use the <b>time Python module</b> to access a set of functions related with the handling of time. For this exercise you can use the <b>time()</b> function, which returns the system time in seconds.
      <a href="https://www.programiz.com/python-programming/time">https://www.programiz.com/python-programming/time</a>
</div>

<div style="background:#946db2;font-family:'arial', monospace; text-align: center; padding: 10px 0; border-radius:10px; width:70%; margin:auto " >
  <span style="font-size:20px;position:relative;color:white; "> Explore </span> <br>
  <div style="background:#d0b3e6;font-size:12px"> 
    For in-depth documentation and examples regarding the use of the PySerial (serial) module, please refer to:
<a href="https://pythonhosted.org/pyserial/shortintro.html">https://pythonhosted.org/pyserial/shortintro.html</a>
</div>

## <div style="color:#00a0e4;">   3. Plotting in Real-Time</div>


Things can gain a totally different meaning when we’re seeing visual representations rather than just numbers going by. In this experiment we will build upon our serial monitor tool, and implement a program that can show a live (interactive) representation of the data streamed by the Arduino.

1. Create a copy of the Python program developed in the previous section

2. Adapt the program to accumulate the values from the analog input channel streamed by the device in a variable of type `list` and plot the variable whenever it is updated

3. Run your program and verify the output; a graphic should appear, but (most likely) only when the data acquisition stops

4. To force the graphic to be redrawn in what is known as interactive mode, use the function `pause(...)` from the **Matplotlib** module (even a very short amount of time such as 0.00001 seconds should make the graphic show up in real time)

5. Run your program and verify the output; an interactive graphic should appear, but (most  likely) in "rainbow" party mode... this happens because the Python graphics library accumulates each time series in the same plot, so what you’re seeing is an overlap of the plots made whenever a new message is received from the Arduino and the data vector updated

6. Change this behaviour by clearing the visualization area prior to plotting the data (you can use the function `cla()` from the **Matplotlib** module for this)

7. Run your program and verify the output; everything should be working as expected

<div style="background:#fbb144;font-family:'arial', monospace; text-align: center; padding: 10px 0; border-radius:10px; width:70%; margin:auto " >
  <span style="font-size:20px;position:relative;color:white; ">  Warning! </span> <br>
  <div style="background:#ffd08a;font-size:12px"> 
     By default Spyder is generally configured to display the graphics inline within the interactive shell. For this exercise <b>you’ll need to have the graphics as a separate window</b>, which can be configured by accessing the Spyder Preferences > IPython Console > Graphics pane and in the Backend option selecting (e.g.) Tkinter. You should need to restart the Spyder IDE for the changes to take effect. 
</div>
</div>
<br>

<div style="background:#48ba57;font-family:'arial', monospace; text-align: center; padding: 10px 0; border-radius:10px; width:70%; margin:auto " >
  <span style="font-size:20px;position:relative;color:white; ">  Note </span> <br>
  <div style="background:#9de3a6;font-size:12px"> 
      You can use the <b>Matplotlib</b> Python module to access a set of functions related with data plotting. For this exercise you can use the <i>plot()</i> function to show a graphic of a data vector. Note that the <i>show()</i> function should be called to display the graphic for the first time.
</div>
</div>
<br>

<div style="background:#946db2;font-family:'arial', monospace; text-align: center; padding: 10px 0; border-radius:10px; width:70%; margin:auto " >
  <span style="font-size:20px;position:relative;color:white; "> Explore </span> <br>
  <div style="background:#d0b3e6;font-size:12px"> 
      For in-depth <b>documentation and examples</b> regarding the use of the plotting functions, please refer to:
<a href="https://matplotlib.org/stable/tutorials/index.html">https://matplotlib.org/stable/tutorials/index.html</a>
</div></div>


## <div style="color:#00a0e4;">   4. My Data Logger </div>


Recording data collected by a given device for later analysis, sharing and/or processing is one of the most fundamental tasks when dealing with biomedical signals. To explore this dimension, in this experiment we will create a program that stores the data streamed by the Arduino in standard CSV (comma-separated values) files.

1. Create a copy of the Python program developed in section 2, *DiY Serial Monitor*

2. Adapt the program to continously store the data streamed by the Arduino in an CSV file, in addition to printing it on the console (as before)

3. Run your program

4. Use a spreadsheet software of your choice to open the resulting file and visually verify the results by creating a chart

<div style="background:#946db2;font-family:'arial', monospace; text-align: center; padding: 10px 0; border-radius:10px; width:70%; margin:auto " >
  <span style="font-size:20px;position:relative;color:white; "> Explore </span> <br>
  <div style="background:#d0b3e6;font-size:12px"> 
      For <b>examples</b> regarding the writing to files in Python, please refer to:
<a href="https://www.geeksforgeeks.org/writing-to-file-in-python/">https://www.geeksforgeeks.org/writing-to-file-in-python/</a>
</div></div>

# III. Explore
<br>
<div class='h1'  style="width:100%; background:#00a0e4;color:#282828;font-family:'arial black'; text-align: center; padding: 7px 0; border-radius: 5px 50px;margin-top:-15px" > </div>

## <div style="color:#00a0e4;">  1. Quiz  </div>

**1.** The Arduino millis() function returns the number of milliseconds since a currently executed program started running. As a result, the time streamed by the firmware of Section II.2. does not begin in 0 when the transmission starts. Adapt the Arduino firmware so that a zero-based timeline is streamed. Using the provided firmware, adapt your Python code so the same effect is obtained.

**2.** Adapt the program created in Section II.2. so instead of printing on the console the data streamed by the Arduino, it computes and prints the `min`, `max`, and `mean` values every 2 seconds of the previous 2 seconds of data streamed. 

**3.** For the program created in Section II.3., see how many data points (aka samples) were accumulated in the list upon completion of the code. Comment the parts of the program that plot the data in real time an see how many samples were accumulated this time. Was there a difference in the number of samples? Explain what happened.
    
**4.** Create a new Python program for bulk reading of the data stored as a result of the experiment performed in Section II.4., and display of a plot (complete with axis labels, title and legend) showing the time plotted against the signal from the function generator. Improve your code so it automatically saves the generated plot into an image file.

##  <div style="color:#00a0e4;">  2. Beyond the Lab </div>

This section is not mandatory and will not influence the evaluation of the lab. These are suplementary exercises designed for those who wish to explore and learn more about Arduino-based systems and Python signal processing.

**1.** Generating a sine wave with the Function Generator, write a Python program that detects the peaks of the signal based on the derivative of the signal or zero crossings.

<img src="https://www.researchgate.net/profile/Bogdan-Kasztenny/publication/321255243/figure/fig8/AS:668326997024778@1536352981536/Zero-crossing-detection-for-frequency-measurement.png" width="500">

<div style="height:100px; background:white;border-radius:10px;text-align:center"> 

<a> <img src="https://github.com/PIA-Group/ScientIST-notebooks/blob/master/_Resources/Images/IT.png?raw=true" alt="it" style=" bottom: 0; width:250px;
    display: inline;
    left: 250px;
    position: absolute;"/> </a>
<img src="https://github.com/PIA-Group/ScientIST-notebooks/blob/master/_Resources/Images/IST.png?raw=true"
         alt="alternate text" 
         style="position: relative;   width:250px;"/>
</div> 

<div style="width: 100%; ">
<div style="background:linear-gradient(to right,#FDC86E,#fbb144);font-family:'arial', monospace; text-align: center; padding: 50px 0; border-radius:10px; height:10px; width:100%; float:left " >
<span style="font-size:12px;position:relative; top:-25px">  Please provide us your feedback <span style="font-size:14px;position:relative;COLOR:WHITE"> <a href="https://forms.gle/C8TdLQUAS9r8BNJM8">here</a>.</span></span> 
<br>
<span style="font-size:17px;position:relative; top:-20px">  Suggestions are welcome! </span> 
</div>

`Contributors: Prof. Hugo Silva; Joana Pinto; Afonso Raposo`